#### Count Encoding
カテゴリ変数を出現回数に置き換える手法で，例えば「赤，青，青，黄」と並んでいれば出現回数は赤は1回，青は2回，黄は1回なので「1, 2, 2, 1」と数値化します．

In [ ]:
cat_cols = list(df.select_dtypes(object).columns)
for col in cat_cols:
    encoder = train[col].value_counts()
    train[f'label_{col}'] = train[col].map(encoder)
    test[f'label_{col}'] = test[col].map(encoder)

#### Label Encoding

単純に数値ラベルに変換する手法で，例えば「赤，青，青，黄」と並んでいれば赤を0，青を1，黄を2と置き換えて「0, 1, 1, 2」と数値化します．次元数を増やさずGBDTなどの決定木に有効なEncodingです．

In [ ]:
from sklearn.preprocessing import LabelEncoder

cat_cols = ["カテゴリー名"]
for col in cat_cols:
    encoder = LabelEncoder()
    encoder.fit(train[col])
    train[f'label_{col}'] = encoder.transform(train[col])
    test[f'label_{col}'] = encoder.transform(test[col])

#### One-hot Encoding

例えばある列に「赤，青，青，黄」と並んでいた時，新しい列として「赤であるか，青であるか，黄色であるか」と水準数分作成して，正しければ1，異なれば0とダミー変数化する手法です．決定木以外のモデルに使われるEncodingです．

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_cols = ["カテゴリー名"]
for col in cat_cols:
    encoder = OneHotEncoder()
    encoder.fit(train[col])
    train[f'label_{col}'] = encoder.transform(train[col])
    test[f'label_{col}'] = encoder.transform(test[col])

#### Target Encoding

カテゴリ変数をそれぞれ目的変数の期待値に置き換える手法です．Cardinality(水準数)が高いほど効果が期待される手法ですが，trainとtestの分布が異なっていたり，学習時のデータの分割に合わせて処理しなければリーク(学習時のスコアが異常に高くなってしまう)してしまう恐れがあります．

Cardinalityが高い場合にGDBTの特性上，Label EncodingよりもTarget Encodingのほうが効率的である。

In [ ]:
from sklearn.model_selection import KFold

def get_kfold(train, n_splits, seed):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    fold_series = []
    for fold, (idx_train, idx_valid) in enumerate(kf.split(train)):
        fold_series.append(pd.Series(fold, index=idx_valid))
    fold_series = pd.concat(fold_series).sort_index()
    return fold_series
    
def get_targetencoding(train, test, folds: pd.Series, col: str):
    for fold in folds.unique():
        idx_train, idx_valid = (folds!=fold), (folds==fold)
        group = train[idx_train].groupby(col)['target'].mean().to_dict()
        train.loc[idx_valid, f'target_{col}'] = train.loc[idx_valid, col].map(group)
    group = train.groupby(col)['target'].mean().to_dict()
    test[f'target_{col}'] = test[col].map(group)
    return train, test

folds = get_kfold(train, 5, 42)
for col in cat_cols:
    train, test = get_targetencoding(train, test, folds, col)